# Stock Options Performance

This project seeks to explore if there is a relationship between stock performance and options buying or selling activity for a given stock.

## Import Trade Data

In [1]:
import pandas as pd
trades = pd.read_csv("https://raw.githubusercontent.com/CollierKing/StockMarket_ShinyDB/master/all_trades.csv")

In [2]:
trades.head()

Ticker  Strike OptionType ActivityType  InitialVolume IS_Flag  \
0    GGP    29.0      Calls       BUYING           1300     NaN   
1    SPY   218.0      Calls      SELLING           2503     NaN   
2   JNPR    22.0      Calls       BUYING            493     NaN   
3   RLGY    30.0      Calls       BUYING            500     NaN   
4    GLD   127.0      Calls       BUYING          10000     NaN   

        TweetTimeStamp   TradeDate TradeTime     ExpDate  startOpen  startLow  \
0  2016-07-12 14:59:17  2016-07-12  14:59:17  2016-07-15      31.09     30.48   
1  2016-07-12 14:58:51  2016-07-12  14:58:51  2016-08-05     214.53    213.43   
2  2016-07-12 14:39:41  2016-07-12  14:39:41  2016-07-15      23.05     22.97   
3  2016-07-12 14:38:01  2016-07-12  14:38:01  2016-08-19      29.34     29.33   
4  2016-07-12 14:29:40  2016-07-12  14:29:40  2016-07-15     128.52    126.99   

   startHigh  startClose  startDayDelt EarningsDate EarningsTime  
0      31.18       31.06     -0.000965   2016-08-01        after  
1     215.30      214.95      0.001958          NaN          NaN  
2      23.29       23.10      0.002169   2016-07-26        after  
3      29.94       29.71      0.012611   2016-08-04       before  
4     128.54      127.15     -0.010660          NaN          NaN

### Trade Scope

In [157]:
trade_date_range = trades.groupby("Ticker").agg({"TradeDate":{"first"}})

In [158]:
trade_date_range.reset_index(inplace=True)
trade_date_range = pd.DataFrame(trade_date_range.values,columns=["Ticker","Date"])

In [160]:
print("Ticker Count:")
print(trade_date_range.shape)

Ticker Count:
(1808, 2)


In [161]:
print("Trade Count:")
print(trades.shape)

Trade Count:
(30947, 17)


We have a little over 1800 traded securities (stocks) with more than 30K trades in our dataset. This analysis will look at stock performance 30 days before and after a given trade, so we will need to download the stock prices for each ticker 30 days prior to the trade and 30 days after the final trade.

# Import Stock Price Data

To import stock price data, we will use the _____ package.

In [168]:
import pandas_datareader.data as web
import fix_yahoo_finance
import time
import datetime

In [169]:
trade_date_range['Date'] = pd.to_datetime(trade_date_range['Date'])
trade_date_range['Date2'] = pd.to_datetime(trade_date_range['Date']).apply(pd.DateOffset(-30))

In [170]:
for idx, row in trade_date_range.iterrows():
#     if idx > 4:
#         break
#     else:
    start_date = trade_date_range['Date2'][idx]
    ticker = trade_date_range['Ticker'][idx]
    try:
        target = web.DataReader(ticker, "yahoo",start=start_date)
        target["ticker"] = ticker
    except:
        continue
    if idx == 0:
        targetAll = target
    else:
        targetAll = pd.concat([targetAll,target])
    if idx % 100 == 0:
        print(idx/len(trade_date_range))
    time.sleep(1)

0.0
0.0553097345133
0.110619469027
0.16592920354
0.221238938053
0.276548672566
0.33185840708
0.442477876106
0.663716814159
0.774336283186
0.829646017699
0.884955752212
0.940265486726


In [226]:
missed_tickers = trade_date_range[~trade_date_range['Ticker'].isin(targetAll['ticker'])]['Ticker']

In [230]:
for idx,ticker in enumerate(missed_tickers):
    try:
        target_missed = web.DataReader(ticker, "yahoo",start="2016-01-01")
        target_missed["ticker"] = ticker
    except:
        continue
    if idx == 0:
        targetAll_missed = target_missed
    else:
        targetAll_missed = pd.concat([targetAll_missed,target_missed])
    time.sleep(1)

In [231]:
targetAll_missed.shape

(92760, 7)

In [233]:
targetAll = pd.concat([targetAll,targetAll_missed])

In [235]:
saveDir = "C:/Users/David/Dropbox/Skills/Python/Projects/Stocks/stock_options_performance/"
targetAll.to_csv(saveDir+"quotes.csv")
trades.to_csv(saveDir+"trades.csv")

## Import from GH

## Calculate Forward Returns

In [238]:
targetAll['5_day_fwd'] = targetAll['Close'].shift(-5)
targetAll['10_day_fwd'] = targetAll['Close'].shift(-10)
targetAll['15_day_fwd'] = targetAll['Close'].shift(-15)
targetAll['20_day_fwd'] = targetAll['Close'].shift(-20)
targetAll['25_day_fwd'] = targetAll['Close'].shift(-25)
targetAll['30_day_fwd'] = targetAll['Close'].shift(-30)

targetAll['5_day_fwd'] = (targetAll['5_day_fwd'] - targetAll['Close'])/targetAll['Close']
targetAll['10_day_fwd'] = (targetAll['10_day_fwd'] - targetAll['Close'])/targetAll['Close']
targetAll['15_day_fwd'] = (targetAll['15_day_fwd'] - targetAll['Close'])/targetAll['Close']
targetAll['20_day_fwd'] = (targetAll['20_day_fwd'] - targetAll['Close'])/targetAll['Close']
targetAll['25_day_fwd'] = (targetAll['25_day_fwd'] - targetAll['Close'])/targetAll['Close']
targetAll['30_day_fwd'] = (targetAll['30_day_fwd'] - targetAll['Close'])/targetAll['Close']
targetAll['Date'] = targetAll.index

In [239]:
targetAll.head()

10_day_fwd  15_day_fwd  20_day_fwd  25_day_fwd  30_day_fwd  \
Date                                                                     
2017-02-14    0.032745    0.020242    0.073626    0.054376    0.054177   
2017-02-15   -0.004649   -0.003681    0.027702    0.030221    0.032933   
2017-02-16    0.022423    0.025767    0.048584    0.044060    0.039929   
2017-02-17   -0.007360    0.015882    0.029440    0.031183    0.030409   
2017-02-21   -0.002726    0.016748    0.025511    0.040701    0.024732   

            5_day_fwd  Adj Close      Close       Date       High        Low  \
Date                                                                           
2017-02-14   0.018853  50.153549  50.389999 2017-02-14  50.680000  50.090000   
2017-02-15  -0.002906  51.377777  51.619999 2017-02-15  51.840000  50.619999   
2017-02-16   0.017703  50.601440  50.840000 2017-02-16  51.680000  50.560001   
2017-02-17   0.000581  51.387737  51.630001 2017-02-17  51.639999  50.580002   
2017-02-21  -0.000974  51.109043  51.349998 2017-02-21  51.799999  51.169998   

                 Open     Volume ticker  
Date                                     
2017-02-14  50.240002  3521100.0      A  
2017-02-15  51.139999  5724500.0      A  
2017-02-16  51.680000  2550400.0      A  
2017-02-17  50.650002  2195500.0      A  
2017-02-21  51.660000  2037000.0      A

In [240]:
trades['TradeDate'] = pd.to_datetime(trades['TradeDate'])

In [241]:
trades_quotes = pd.merge(trades,targetAll, left_on=  ['Ticker', 'TradeDate'],
                   right_on= ['ticker', 'Date'], 
                   how = 'left')
trades_quotes.sort_values(by=["Ticker","TradeDate"],inplace=True)
trades_quotes = trades_quotes[["Ticker","TradeDate","OptionType","ActivityType",
                              "5_day_fwd","10_day_fwd","15_day_fwd","20_day_fwd","25_day_fwd","30_day_fwd"]]
trades_quotes = trades_quotes[pd.notnull(trades_quotes['5_day_fwd'])]

Ticker  TradeDate OptionType ActivityType  5_day_fwd  10_day_fwd  \
933     ACIA 2017-01-10      Calls       BUYING        NaN    0.110706   
934     ACIA 2017-03-08       Puts       BUYING        NaN         NaN   
935     ACIA 2017-03-08       Puts       BUYING        NaN         NaN   
936     ACIA 2017-03-21       Puts      SELLING        NaN         NaN   
937     ACIA 2017-08-08       Puts       BUYING        NaN         NaN   
940      ACM 2016-07-12      Calls      SELLING        NaN         NaN   
941      ACM 2016-07-12      Calls      SELLING        NaN         NaN   
942      ACM 2016-08-16       Puts       BUYING        NaN         NaN   
943      ACM 2016-08-31       Puts       BUYING        NaN         NaN   
944      ACM 2016-09-20      Calls      SELLING        NaN         NaN   
945      ACM 2016-09-30      Calls       BUYING        NaN         NaN   
946      ACM 2016-11-29       Puts       BUYING        NaN         NaN   
947      ACM 2016-11-30       Puts      SELLING        NaN         NaN   
1041    ADPT 2016-11-01       Puts       BUYING        NaN         NaN   
1042    ADPT 2017-02-14       Puts      SELLING        NaN         NaN   
1043    ADPT 2017-03-09       Puts       BUYING        NaN         NaN   
1287     AHS 2016-10-18      Calls       BUYING        NaN         NaN   
1288     AHS 2016-10-18      Calls      SELLING        NaN         NaN   
1351    AIRM 2016-10-31       Puts       BUYING        NaN         NaN   
1352    AIRM 2016-11-03       Puts       BUYING        NaN         NaN   
1353    AIRM 2016-11-04      Calls      SELLING        NaN         NaN   
1489     ALJ 2016-08-11      Calls       BUYING        NaN         NaN   
1558     ALR 2016-07-28       Puts       BUYING        NaN         NaN   
1559     ALR 2016-11-08       Puts       BUYING        NaN         NaN   
1560     ALR 2016-12-28       Puts      SELLING        NaN         NaN   
1561     ALR 2017-02-03       Puts      SELLING        NaN         NaN   
1562     ALR 2017-02-07       Puts       BUYING        NaN         NaN   
1563     ALR 2017-05-16       Puts       BUYING        NaN         NaN   
2020     APO 2017-09-27      Calls      SELLING        NaN         NaN   
2021     APO 2017-09-28      Calls      SELLING        NaN         NaN   
...      ...        ...        ...          ...        ...         ...   
30592   YHOO 2017-02-22       Puts       BUYING        NaN         NaN   
30593   YHOO 2017-02-22       Puts       BUYING        NaN         NaN   
30594   YHOO 2017-03-16      Calls       BUYING        NaN         NaN   
30595   YHOO 2017-03-23      Calls      SELLING        NaN         NaN   
30596   YHOO 2017-03-23      Calls       BUYING        NaN         NaN   
30597   YHOO 2017-04-05      Calls       BUYING        NaN         NaN   
30598   YHOO 2017-04-05      Calls       BUYING        NaN         NaN   
30599   YHOO 2017-04-06      Calls       BUYING        NaN         NaN   
30600   YHOO 2017-04-07      Calls      SELLING        NaN         NaN   
30601   YHOO 2017-05-05      Calls       BUYING        NaN         NaN   
30602   YHOO 2017-05-09      Calls       BUYING        NaN         NaN   
30603   YHOO 2017-05-09      Calls       BUYING        NaN         NaN   
30604   YHOO 2017-05-12      Calls       BUYING        NaN         NaN   
30605   YHOO 2017-05-16      Calls      SELLING        NaN         NaN   
30606   YHOO 2017-05-16      Calls      SELLING        NaN         NaN   
30607   YHOO 2017-05-22      Calls      SELLING        NaN         NaN   
30608   YHOO 2017-05-24      Calls      SELLING        NaN         NaN   
30609   YHOO 2017-05-25       Puts       BUYING        NaN         NaN   
30610   YHOO 2017-06-06       Puts      SELLING        NaN         NaN   
30611   YHOO 2017-06-07      Calls       BUYING        NaN         NaN   
30612   YHOO 2017-06-08       Puts      SELLING        NaN         NaN   
30613   YHOO 2017-06-09       Puts      SELLING        NaN         NaN   
30614   YH